In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.5MB/s 


In [ ]:
!pip install datasets

     |████████████████████████████████| 266kB 9.8MB/s 
     |████████████████████████████████| 245kB 52.6MB/s 
     |████████████████████████████████| 122kB 59.4MB/s 


In [ ]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8MB 8.8MB/s 
     |████████████████████████████████| 2.9MB 41.2MB/s 
     |████████████████████████████████| 901kB 42.8MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, \
    BertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel,  
    AlbertModel, AlbertTokenizer
from transformers import CamembertModel, CamembertTokenizer, 
    CamembertForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from transformers import TrainingArguments
from datasets import load_metric


def train_semantical_model():
    batch_size = 8
    
    raw_datasets = load_dataset("/content/drive/MyDrive/data/src/bert_dataset.py")
    tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
    model = CamembertForSequenceClassification.from_pretrained("camembert-base",
                                                          num_labels=9)
    # print(len(raw_datasets))
    def tokenize_function(examples):
        tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

    tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets.set_format("torch")

    full_train_dataset = tokenized_datasets["train"]
    full_eval_dataset = tokenized_datasets["test"]
    small_train_dataset = full_train_dataset
    small_eval_dataset = full_eval_dataset
    method = "PYTORCH_API" 

    if method == "PYTORCH_API":
        metric = load_metric("accuracy")

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            return metric.compute(predictions=predictions, references=labels)

        training_args = TrainingArguments(evaluation_strategy="epoch", 
                                          no_cuda=False, num_train_epochs=10,
                                          output_dir="/content/drive/MyDrive/data/checkpoints", 
                                          per_device_train_batch_size=batch_size,
                                          per_device_eval_batch_size=batch_size)
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=small_train_dataset,
            eval_dataset=small_eval_dataset,
            compute_metrics=compute_metrics,
        )
        
        trainer.train()

        trainer.evaluate()

    elif method == "PYTORCH":          
      train_dataloader = DataLoader(small_train_dataset, shuffle=True,
                                    batch_size=batch_size)
      eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

      optimizer = AdamW(model.parameters(), lr=5e-5)

      num_epochs = 3
      num_training_steps = num_epochs * len(train_dataloader)
      lr_scheduler = get_scheduler(
          "linear",
          optimizer=optimizer,
          num_warmup_steps=0,
          num_training_steps=num_training_steps
      )

      device = torch.device(
          "cuda") if torch.cuda.is_available() else torch.device("cpu")
      model.to(device)

      progress_bar = tqdm(range(num_training_steps))

      model.train()
      for epoch in range(num_epochs):
          loss = None
          for idx, batch in enumerate(train_dataloader):
              
              batch = {k: v.to(device) for k, v in batch.items()}
              outputs = model(**batch)
              loss = outputs.loss
              loss.backward()

              optimizer.step()
              lr_scheduler.step()
              optimizer.zero_grad()
              progress_bar.update(1)

              if idx % 5 == 0: 
                  torch.save({
                      'epoch': epoch,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': loss,

                  }, os.path.join("/content/drive/MyDrive/data", "checkpoints", 
                                  "BERT_MINISTRY_MODEL_EPOCH_" + str(
                      epoch) + ".pth"))


      metric = load_metric("accuracy")
      model.eval()

      for batch in eval_dataloader:
          batch = {k: v.to(device) for k, v in batch.items()}
          with torch.no_grad():
              outputs = model(**batch)

          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          metric.add_batch(predictions=predictions, references=batch["labels"])

      metric.compute()


train_semantical_model()

Using custom data configuration default


Dataset parliamentary_questions_dataset downloaded and prepared to /root/.cache/huggingface/datasets/parliamentary_questions_dataset/default/0.0.0/473f7578d99dc5bfbe5e1ea86c1d4f6106aa324f439509793ad2bb1b7e0da359. Subsequent calls will reuse this data.


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.446000,0.993021,0.695408,61.059600,32.100000
2,0.832800,1.002421,0.693367,61.213000,32.019000
3,0.645500,1.006248,0.719388,61.226900,32.012000
4,0.520800,1.158823,0.713776,61.142300,32.056000
5,0.393500,1.276462,0.716327,61.086200,32.086000


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-081cba77a9ab>", line 137, in <module>
    train_semantical_model()
  File "<ipython-input-5-081cba77a9ab>", line 68, in train_semantical_model
    trainer.train()
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 888, in train
    tr_loss += self.training_step(model, inputs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1267, in training_step
    loss.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 255, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 149, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
Keyboar

KeyboardInterrupt: ignored